# Investigate Bad Filters

In our previous notebook, we observed that there were four filters that didn't seem to respond to very much. Let's see what happens when we just drop the filters altogether. Do we still get good results? Or are these filters actually useful and the network doesn't work without them?

Since our network was trained on ImageNet, we'll test it out against the CIFAR-10.

In [69]:
from fastai.vision import *

## Basic Training

First let's sanity check that we can train on CIFAR.

For pretrained models, fastai will freeze the body of the network and only train the head.

In [70]:
path = untar_data(URLs.CIFAR)

In [100]:
def trainLearner():
    data = ImageDataBunch.from_folder(path, valid='test', bs=128)
    learner = cnn_learner(data, models.resnet50, metrics=accuracy, pretrained=True)
    learner.fit_one_cycle(10, max_lr=1e-2)
    acc = learner.recorder.metrics[-1]
    return acc

In [101]:
acc1 = trainLearner()
acc2 = trainLearner()
acc3 = trainLearner()

print("Avg accuracy:", np.mean((acc1, acc2, acc3)))

epoch,train_loss,valid_loss,accuracy,time
0,1.244443,1.088225,0.646300,00:16
1,0.927991,0.885325,0.705900,00:16
2,0.892676,0.846013,0.707900,00:16
3,0.736301,0.833539,0.718200,00:17
4,0.701119,0.698902,0.760100,00:17
5,0.644356,0.635338,0.777400,00:17
6,0.547222,0.578339,0.798900,00:17
7,0.469942,0.541901,0.811800,00:17
8,0.406963,0.547133,0.816900,00:17
9,0.345667,0.550512,0.813100,00:17


epoch,train_loss,valid_loss,accuracy,time
0,1.228941,1.105009,0.645500,00:16
1,0.967721,0.965166,0.680600,00:16
2,0.877300,1.211801,0.687200,00:17
3,0.788042,0.860623,0.707100,00:17
4,0.722246,0.689881,0.760500,00:16
5,0.614706,0.627281,0.786700,00:17
6,0.542406,0.596784,0.796800,00:17
7,0.466417,0.550768,0.809800,00:17
8,0.397846,0.545762,0.816500,00:17
9,0.365790,0.545271,0.817700,00:17


epoch,train_loss,valid_loss,accuracy,time
0,1.236661,1.014586,0.646900,00:16
1,0.906971,0.845509,0.711700,00:17
2,0.914414,0.834185,0.715500,00:17
3,0.793425,0.737999,0.747900,00:16
4,0.703719,0.727799,0.753700,00:16
5,0.630391,0.697964,0.760500,00:17
6,0.537599,0.582982,0.797900,00:17
7,0.453328,0.557602,0.812800,00:16
8,0.403366,0.551810,0.816400,00:17
9,0.360014,0.558081,0.818500,00:16


Avg accuracy: 0.81643337


## Training Without "Bad" Filters

Next we'll try the same exercise, but we'll completely omit the "bad" filters. If they're truly doing nothing, then we should expect to get comparably performance without them.

In [104]:
def trainLearnerButDropFiltersFromFirstConv(filter_indexes):
    data = ImageDataBunch.from_folder(path, valid='test', bs=128)
    learner = cnn_learner(data, models.resnet50, metrics=accuracy, pretrained=True)
    
    # drop any filters for conv1 that are passed in as `filter_indexes`
    filters = learner.model[0][0].weight
    for idx in filter_indexes:
        filters[idx].mul_(0)
        
    learner.fit_one_cycle(10, max_lr=1e-2)
    acc = learner.recorder.metrics[-1]
    return acc

In [105]:
acc1 = trainLearnerButDropFiltersFromFirstConv([13, 42, 52, 58])
acc2 = trainLearnerButDropFiltersFromFirstConv([13, 42, 52, 58])
acc3 = trainLearnerButDropFiltersFromFirstConv([13, 42, 52, 58])

print("Avg accuracy:", np.mean((acc1, acc2, acc3)))

epoch,train_loss,valid_loss,accuracy,time
0,1.247845,1.080709,0.642400,00:15
1,1.050161,1.262804,0.593600,00:15
2,0.972370,0.905316,0.682200,00:16
3,0.760521,0.799287,0.728100,00:16
4,0.693087,0.742944,0.765800,00:16
5,0.627960,0.647821,0.771800,00:16
6,0.563726,0.584559,0.795000,00:16
7,0.474862,0.553025,0.806700,00:15
8,0.400794,0.550689,0.813200,00:16
9,0.360150,0.559005,0.812100,00:16


epoch,train_loss,valid_loss,accuracy,time
0,1.234343,0.999440,0.661800,00:16
1,1.034868,1.219926,0.668000,00:16
2,0.813259,0.976878,0.710800,00:15
3,0.749725,0.685710,0.761600,00:16
4,0.660100,0.686995,0.763600,00:18
5,0.611054,0.607105,0.792000,00:16
6,0.536444,0.790822,0.800000,00:16
7,0.456535,0.540729,0.814200,00:16
8,0.390104,0.542062,0.817400,00:17
9,0.349075,0.548315,0.817000,00:17


epoch,train_loss,valid_loss,accuracy,time
0,1.276838,1.220675,0.644500,00:17
1,1.007382,0.906003,0.685200,00:16
2,0.906013,0.984407,0.665000,00:16
3,0.758477,0.688748,0.758500,00:16
4,0.682145,0.675771,0.763600,00:16
5,0.594602,0.620319,0.784300,00:16
6,0.535475,0.565233,0.801500,00:15
7,0.457108,0.536073,0.814600,00:16
8,0.383365,0.532336,0.821500,00:16
9,0.335319,0.539524,0.822300,00:16


Avg accuracy: 0.81713337


So accuracy is roughly unchanged. (I think it's likely just chance that it increased, in my earlier results it seemed consistently lower...)

## Training without random filters

So accuracy is basically unchanged. Is this because the filters we dropped are useless or is it just a property of neural networks that they're pretty good at recovering after brain surgery?

Let's try the same experiment, but we'll drop random filters that are not in our list of "bad" filters.

In [115]:
def get_filters_to_drop():
    bad_filter_idx = [13, 42, 52, 58]

    filters_to_drop = []

    while len(filters_to_drop) < 4:
        rand_idx = np.random.randint(0, 64)
        if rand_idx not in bad_filter_idx and rand_idx not in filters_to_drop:
            filters_to_drop.append(rand_idx)
            
    return filters_to_drop    

In [116]:
filters_to_drop = get_filters_to_drop()
print("Dropping", filters_to_drop)
acc1 = trainLearnerButDropFiltersFromFirstConv(filters_to_drop)
filters_to_drop = get_filters_to_drop()
print("Dropping", filters_to_drop)
acc2 = trainLearnerButDropFiltersFromFirstConv(filters_to_drop)
filters_to_drop = get_filters_to_drop()
print("Dropping", filters_to_drop)
acc3 = trainLearnerButDropFiltersFromFirstConv(filters_to_drop)

print("Avg accuracy:", np.mean((acc1, acc2, acc3)))

Dropping [32, 1, 5, 17]


epoch,train_loss,valid_loss,accuracy,time
0,1.286040,1.217069,0.620300,00:16
1,1.005321,0.995700,0.678900,00:17
2,0.982823,1.169003,0.699400,00:17
3,0.747405,0.811925,0.718700,00:16
4,0.695665,0.728269,0.749100,00:17
5,0.620434,0.628402,0.781500,00:17
6,0.532214,0.598709,0.794700,00:17
7,0.458927,0.560244,0.809300,00:16
8,0.395093,0.549830,0.815300,00:17
9,0.357761,0.556244,0.815700,00:17


Dropping [33, 34, 24, 57]


epoch,train_loss,valid_loss,accuracy,time
0,1.254056,1.066417,0.641300,00:17
1,0.997040,1.005947,0.668800,00:17
2,1.059929,0.968771,0.663500,00:16
3,0.799531,0.831358,0.719800,00:17
4,0.693111,0.722479,0.746200,00:17
5,0.627966,0.635978,0.781200,00:17
6,0.548624,0.584080,0.802100,00:17
7,0.482482,0.553478,0.808700,00:17
8,0.391560,0.554136,0.815400,00:17
9,0.358610,0.562674,0.816300,00:17


Dropping [55, 47, 27, 48]


epoch,train_loss,valid_loss,accuracy,time
0,1.244725,1.107273,0.640600,00:17
1,1.074177,1.498895,0.652400,00:17
2,0.916429,0.863429,0.711900,00:17
3,0.765995,0.748045,0.742300,00:16
4,0.686289,0.773112,0.730800,00:16
5,0.633192,0.660814,0.769100,00:17
6,0.535396,0.583198,0.794200,00:16
7,0.469384,0.558696,0.808000,00:16
8,0.407914,0.559097,0.809300,00:16
9,0.359167,0.566663,0.808200,00:16


Avg accuracy: 0.8134


So we see a small decrease. Not enough for me to say anything conclusively. 

To be honest, after running this experiment, we haven't really learned that much. But we've at least crossed off this line of investigation for now.

A potential followup would be to run this same experiment on a different (maybe more challenging) dataset.